# L2: Multi-agent Customer Support Automation

In this lesson, you will learn about the six key elements which help make Agents perform even better:
- Role Playing
- Focus
- Tools
- Cooperation
- Guardrails
- Memory

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, API and LLM

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

openai_api_key = os.environ["OPENAI_KEY"]
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

## Role Playing, Focus and Cooperation

In [4]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True
)

- By not setting `allow_delegation=False`, `allow_delegation` takes its default value of being `True`.
- This means the agent _can_ delegate its work to another agent which is better suited to do a particular task. 

In [5]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True
)

* **Role Playing**: Both agents have been given a role, goal and backstory.
* **Focus**: Both agents have been prompted to get into the character of the roles they are playing.
* **Cooperation**: Support Quality Assurance Agent can delegate work back to the Support Agent, allowing for these agents to work together.

## Tools, Guardrails and Memory

### Tools

- Import CrewAI tools

In [12]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

### Possible Custom Tools
- Load customer data
- Tap into previous conversations
- Load data from a CRM
- Checking existing bug reports
- Checking existing feature requests
- Checking ongoing tickets
- ... and more

- Some ways of using CrewAI tools.

```Python
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
```

- Instantiate a document scraper tool.
- The tool will scrape a page (only 1 URL) of the CrewAI documentation.

In [13]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

##### Different Ways to Give Agents Tools

- Agent Level: The Agent can use the Tool(s) on any Task it performs.
- Task Level: The Agent will only use the Tool(s) when performing that specific Task.

**Note**: Task Tools override the Agent Tools.

### Creating Tasks
- You are passing the Tool on the Task Level.

In [14]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

- `quality_assurance_review` is not using any Tool(s)
- Here the QA Agent will only review the work of the Support Agent

In [15]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


### Creating the Crew

#### Memory
- Setting `memory=True` when putting the crew together enables Memory.

In [21]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=True
)

### Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

#### Guardrails
- By running the execution below, you can see that the agents and the responses are within the scope of what we expect from them.

In [18]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

 
[2025-03-14 23:21:00][🚀 CREW 'CREW' STARTED, 5E146A34-CF4C-43AD-9B13-FDE24D8CE232]: 2025-03-14 23:21:00.793119
 
[2025-03-14 23:21:00][📋 TASK STARTED: DEEPLEARNINGAI JUST REACHED OUT WITH A SUPER IMPORTANT ASK:
I NEED HELP WITH SETTING UP A CREW AND KICKING IT OFF, SPECIFICALLY HOW CAN I ADD MEMORY TO MY CREW? CAN YOU PROVIDE GUIDANCE?

ANDREW NG FROM DEEPLEARNINGAI IS THE ONE THAT REACHED OUT. MAKE SURE TO USE EVERYTHING YOU KNOW TO PROVIDE THE BEST SUPPORT POSSIBLE.YOU MUST STRIVE TO PROVIDE A COMPLETE AND ACCURATE RESPONSE TO THE CUSTOMER'S INQUIRY.]: 2025-03-14 23:21:00.805120
 
[2025-03-14 23:21:02][🤖 AGENT 'SENIOR SUPPORT REPRESENTATIVE' STARTED TASK]: 2025-03-14 23:21:02.059729
# Agent: Senior Support Representative
## Task: DeepLearningAI just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Andrew Ng from DeepLearningAI is the one that reached out. Make sure

- Display the final result as Markdown.

In [19]:
from IPython.display import Markdown
Markdown(result.raw)

Hi Andrew! 😊 

Thanks for reaching out about adding memory to your Crew in CrewAI. Exciting times ahead! Let’s dive into how you can enhance your agents’ capabilities by implementing memory. Here’s a detailed guide that should help you out:

1. **Define Memory**: Think of memory as a superpower for your agents! It enables them to retain information from their past interactions, meaning they can provide smarter, more contextually relevant responses over time. This is crucial for improving performance and user engagement. 

2. **Implementation**: To get started, you'll want to define what specific information should be retained. This can include user interactions, the decisions your agents make, or any noteworthy outcomes from previous tasks. You can create structured data storage that will keep this information, making it easily retrievable when needed.

3. **Agent Configuration**: As you set up your agents in the Crew, assign them roles that let them utilize this new memory system. It's essential to include parameters in their configurations that specify how they will use their memory, ensuring they can leverage this resource effectively.

4. **Memory Access**: Make sure your agents have access to their stored memory when they need it. This often means implementing functions that allow them to call upon their accumulated knowledge while making decisions, enhancing their responses and overall interaction quality.

5. **Testing and Evaluation**: Once you've integrated memory, it’s a great idea to run some tests to ensure everything is functioning smoothly. Observe how your agents utilize their memory in real-time interactions, and refine your memory storage and retrieval processes based on what you learn. This might help you pinpoint any adjustments needed for optimal performance.

6. **Continuous Improvement**: Remember, memory management isn’t a set-it-and-forget-it task! As your Crew scales and adapts, regularly review how memory is being handled. Update your mechanisms for storing, updating, and deleting memory to cater to evolving requirements, ensuring that your agents stay sharp and effective.

By following these steps, you’re on your way to building a more dynamic Crew that can learn and adapt while working collaboratively and efficiently. If you have any more questions or need specific examples to get you moving, just give me a shout! I’m here to help! 

Best,  
[Your Name]  
CrewAI Support Team